In [1]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr

import yfinance as yf
yf.pdr_override()

In [5]:
tickers = ["GOOG", "MMM", "TSLA"]

data = pdr.get_data_yahoo(tickers, start="2015-01-01", end="2020-09-22")
spy_data = pdr.get_data_yahoo("SPY", start="2015-01-01", end="2020-09-22")

In [11]:
daily_returns = data["Adj Close"].pct_change()
spy_daily = spy_data["Adj Close"].pct_change()

daily_returns.dropna(inplace=True)
spy_daily.dropna(inplace=True)

In [16]:
goog_spy = pd.concat([daily_returns["GOOG"], spy_daily], axis=1)
goog_spy.cov()

,GOOG,Adj Close
GOOG,0.000287,0.000143
Adj Close,0.000143,0.000135


In [17]:
beta = goog_spy.cov()["GOOG"]["Adj Close"]/goog_spy["GOOG"].var()
beta

0.49964692441227415

In [20]:
from scipy import stats

model = stats.linregress(goog_spy["GOOG"].values, goog_spy["Adj Close"].values)
model[0]

4.719197204376501e-05

In [43]:
weights = np.random.rand(len(tickers))
weights = weights/weights.sum()
weights

array([0.41239362, 0.04593285, 0.54167353])

In [44]:
port_returns = (daily_returns * weights).sum(axis=1)
port_returns

Date
2015-01-05   -0.032405
2015-01-06   -0.006980
2015-01-07   -0.001220
2015-01-08    0.001554
2015-01-09   -0.016089
                ...   
2020-09-16   -0.014319
2020-09-17   -0.028577
2020-09-18    0.013393
2020-09-21   -0.001492
2020-09-22   -0.020164
Length: 1440, dtype: float64

In [45]:
model = stats.linregress(port_returns.values, spy_daily.values)
model[0]

0.3125862625466254

In [49]:
port_returns.mean()

0.001540823791646736

In [50]:
capm = 0.02 + (model[0]*(spy_daily.mean() - 0.02))
capm

0.01389702270901182

In [52]:
cm = daily_returns.cov()
cm

Symbols,GOOG,MMM,TSLA
Symbols,,,
GOOG,0.000287,0.000122,0.000208
MMM,0.000122,0.000222,0.000114
TSLA,0.000208,0.000114,0.001195


In [54]:
sigma = np.sqrt(np.dot(weights.T, np.dot(cm, weights)))
sigma

0.022436898503174627

In [56]:
sharpe = (port_returns.mean()-0.02)/sigma
sharpe

-0.8227151451320891

In [62]:
(daily_returns["GOOG"].mean())/daily_returns["GOOG"].std() * np.sqrt(252)

0.8038886641674595

In [59]:
(daily_returns["TSLA"].mean())/daily_returns["TSLA"].std() * np.sqrt(252)

0.9976522370404327

In [ ]:
def capm(tickers, start, end, weights=None):
    
    if weights is None:
        weights = np.random.rand(len(tickers))
        weights = weights/weights.sum()
    else:
        weights = weights
        
    data = pdr.get_data_yahoo(tickers, start=start, end=end)
    spy_data = pdr.get_data_yahoo("SPY", start=start, end=end)
    
    
    
    return e_r, volatility, weights